<a href="https://colab.research.google.com/github/alexanderkell/TensorFlow_projects/blob/master/notebooks/Yelp/2.0-ajmk-BERT-generate-reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import tensorflow as tf
%pip install datasets
%pip install transformers
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModel
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.9 MB/s eta 0:00:00


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 13 09:15:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# load yelp review dataset
dataset = load_dataset('yelp_polarity', split='train')
dataset

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})

In [6]:
dataset_split = dataset.train_test_split(test_size=0.2)

In [7]:
dataset_split['train'][0]

{'text': "Stopped by for their grand opening last night and had a great night. Never been to a PT's before so I figured this was a perfect opportunity to check it out, even though it was far SW. Apparently this place used to be an O'Aces tavern, but you would've never guessed it because the inside looked brand new and was not smoky at all. We arrived at 7 pm, about an hour late for the event and it was already packed inside! So packed that it took a while to squeeze into the bar area to place drink orders. Eventually we were able to track down a waitress to take care of us the rest of the night, otherwise there would've been a lot of waiting at the bar!\\n\\nThey had a couple specials going on including $2 Stellas and $1 vodka or tequila shots...which were kinda gross, but hey it was only a buck! The gf had a couple of signature cocktails, which were full price but wasn't bad at $6-7.50 each. There were a couple of Stella girls promoting their beer and handing out gifts to people drink

In [8]:
dataset_split = dataset_split.flatten()
dataset_split['train'][0]

{'text': "Stopped by for their grand opening last night and had a great night. Never been to a PT's before so I figured this was a perfect opportunity to check it out, even though it was far SW. Apparently this place used to be an O'Aces tavern, but you would've never guessed it because the inside looked brand new and was not smoky at all. We arrived at 7 pm, about an hour late for the event and it was already packed inside! So packed that it took a while to squeeze into the bar area to place drink orders. Eventually we were able to track down a waitress to take care of us the rest of the night, otherwise there would've been a lot of waiting at the bar!\\n\\nThey had a couple specials going on including $2 Stellas and $1 vodka or tequila shots...which were kinda gross, but hey it was only a buck! The gf had a couple of signature cocktails, which were full price but wasn't bad at $6-7.50 each. There were a couple of Stella girls promoting their beer and handing out gifts to people drink

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
bert = TFAutoModel.from_pretrained("distilgpt2")

def preprocess_function(examples):
    return tokenizer(["".join(x) for x in examples["text"]], truncation=True)

tokenized_yelp = dataset_split.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset_split["train"].column_names,
)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/328M [00:00<?, ?B/s]

...

#3:   0%|          | 0/112 [00:00<?, ?ba/s]

#0:   0%|          | 0/28 [00:00<?, ?ba/s]

#2:   0%|          | 0/28 [00:00<?, ?ba/s]

#3:   0%|          | 0/28 [00:00<?, ?ba/s]

#1:   0%|          | 0/28 [00:00<?, ?ba/s]

In [11]:
block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
lm_dataset = tokenized_yelp.map(group_texts, batched=True, num_proc=5)

#0:   0%|          | 0/90 [00:00<?, ?ba/s]

#2:   0%|          | 0/90 [00:00<?, ?ba/s]

#1:   0%|          | 0/90 [00:00<?, ?ba/s]

#3:   0%|          | 0/90 [00:00<?, ?ba/s]

#4:   0%|          | 0/90 [00:00<?, ?ba/s]

#0:   0%|          | 0/23 [00:00<?, ?ba/s]

#1:   0%|          | 0/23 [00:00<?, ?ba/s]

#2:   0%|          | 0/23 [00:00<?, ?ba/s]

#4:   0%|          | 0/23 [00:00<?, ?ba/s]

#3:   0%|          | 0/23 [00:00<?, ?ba/s]

In [13]:
from transformers import TFAutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling

model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [14]:
tokenizer.pad_token = tokenizer.eos_token

tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    lm_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
# see first 5 tokenized reviews
tf_train_set.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 128), dtype=tf.int64, name=None))>

In [19]:
not_tuned_model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

not_tuned_model.compile()
not_tuned_model.evaluate(tf_test_set)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


9488/9488 [==============================] - 167s 17ms/step - loss: 4.3087


4.308733940124512

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

model.compile()
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


 6664/37994 [====>.........................] - ETA: 24:15 - loss: 4.2645

In [ ]:
from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

generator("I love this restaurant")